In [ ]:
import imp
import torch
import PIL.Image
import os
import torchvision
import numpy as np
import random

# globals

MainModel = imp.load_source('MainModel', '/home/kylesargent/resnet50_128_pytorch/resnet50_128_pytorch.py') 
model = torch.load('/home/kylesargent/resnet50_128_pytorch/resnet50_128_pytorch.pth')
model.cuda()

DATASET_PATH = '/home/kylesargent/test/'

batch_size = 16
alpha = .01

In [ ]:
import load_and_transform
reload(load_and_transform)
from load_and_transform import transform_image, get_image_list, load_image_batch, pairwise_distances

In [ ]:
# data preprocessing

import random

images = get_image_list()
np.random.shuffle(images)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(images[:, 1])

labels = le.transform(images[:, 1])

len(labels)

In [ ]:
import torch.nn as nn
from torch.autograd import Variable
import torch
from torch.autograd import Variable
cluster_model = (nn.Linear(128, 128)).cuda()

In [ ]:
def get_repeated_indices(records_array):
    # thanks stack exchange
    idx_sort = np.argsort(records_array)
    sorted_records_array = records_array[idx_sort]
    vals, idx_start, count = np.unique(sorted_records_array, return_counts=True,
                                    return_index=True)
    res = np.split(idx_sort, idx_start[1:])
    vals = vals[count > 1]
    return [i for l in filter(lambda x: x.size > 1, res) for i in l]

get_repeated_indices(np.array([1, 2, 3, 2, 2, 3]))

In [ ]:
# training loop
batch_size = 64
lr = .001
train_frac = 1

from tqdm import tqdm

torch.cuda.empty_cache()

from torch.optim import Adam


optimizer = Adam(cluster_model.parameters(), lr=lr)

for _ in range(2):
    for i in tqdm(range(int(len(images) * train_frac) // batch_size)):
        cluster_model.weight.data = cluster_model.weight / torch.norm(cluster_model.weight)

        batch_labels = np.array(labels[i:i + batch_size])

        # repeats = get_repeated_indices(batch_labels)
    
        if len(repeats) > 0:
            # batch_labels = batch_labels[repeats]
            batch_labels = torch.from_numpy(batch_labels)

            batch_images = images[:, 0][i:i + batch_size]
            # batch_images = batch_images[repeats]
            
            batch_images = load_image_batch(batch_images)

            with torch.no_grad():
                batch_features = model(batch_images)

            batch_features = Variable(batch_features.squeeze())
            batch_codes = cluster_model(batch_features)

            M = pairwise_distances(batch_codes, batch_codes)

            # delta[i][j] is 1 when code i and code j are the same person
            delta = (batch_labels.unsqueeze(0) - batch_labels.unsqueeze(1)) == 0
            delta = delta.type('torch.FloatTensor').cuda() - torch.eye(len(batch_labels)).cuda()

            # print(torch.sum(delta))
            # print(len(repeats))
            print("\n")
            
            loss = torch.sum(delta*M)

            # print(loss)
            # print(torch.sum(delta))
            # print(len(batch_labels) - len(torch.unique(batch_labels)))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [ ]:
# make a miniature dataset

n_people = 10
n_images_per_person = 10

images = get_image_list(n_people, n_images_per_person)
np.random.shuffle(images)
labels = le.transform(images[:, 1])

# chinese restaurant process

from scipy.stats import multivariate_normal

cluster_ids = []
cluster_codes = []
sigma = .01
theta = 1.
alpha = .5
epsilon = .5

batch_size = 1

predicted_labels = []
for i in range(len(images)):
    batch_labels = torch.from_numpy(labels[i:i + batch_size])
    batch_images = load_image_batch(images[:, 0][i:i + batch_size])

    with torch.no_grad():
        batch_features = model(batch_images)

    batch_features = Variable(batch_features.squeeze())
    batch_code = cluster_model(batch_features).cpu().data.numpy()
    
    
    if i == 0:
        # set up first cluster
        cluster_ids = [[0]]
        cluster_codes = [[batch_code]]
        cluster_centers = [batch_code]
        predicted_labels += [0]
    else:
        
        
        # b = len(ks)
        cluster_priors = map(lambda ks: (len(ks) - alpha)/(i + theta), cluster_ids)
        
        B = len(cluster_centers)
        cluster_priors += [(B*alpha + theta)/(i + theta)]
        assert(np.abs(sum(cluster_priors) - 1) < 1e-5)
        
        cluster_likelihoods = [
            multivariate_normal.logpdf(
                batch_code, mean=cluster_center, cov=np.eye(128)*sigma
            ) 
            for cluster_center in cluster_centers
        ]
        
        cluster_likelihoods += [
            multivariate_normal.logpdf(
                batch_code + epsilon, mean=batch_code, cov=np.eye(128)*sigma
            ) 
        ]
        
        print("ids", cluster_ids)
        print("priors", cluster_priors)
        print("likelihoods", cluster_likelihoods)
        
        assert(len(cluster_likelihoods) == len(cluster_priors))
        
        cluster_maps = [cluster_likelihood + np.log(cluster_priors[e]) for e, cluster_likelihood in enumerate(cluster_likelihoods)]
        cluster_map = np.argmax(cluster_maps)
        predicted_labels += [cluster_map]    
            
        if cluster_map == len(cluster_ids):
            cluster_ids += [[i]]
            cluster_codes += [[batch_code]]
            cluster_centers += [batch_code]
        else:
            nk = len(cluster_ids[cluster_map])
            
            cluster_ids[cluster_map] += [i]
            cluster_codes[cluster_map] += [batch_code]
            cluster_centers[cluster_map] = (nk*cluster_centers[cluster_map] + batch_code)/(nk+1.)
            
        

In [ ]:
from sklearn import metrics

metrics.adjusted_mutual_info_score(labels, predicted_labels)  

In [ ]:
# an experiment to verify that applying the cluster layer separates specific persons better

n_people = 5
n_images_per_person = 5

images = get_image_list(n_people, n_images_per_person)
labels = le.transform(images[:, 1])

import itertools
from tqdm import tqdm

l1 = 0
c1 = 0
l1p = 0
c1p = 0

l2 = 0
c2 = 0
l2p = 0
c2p = 0

for (image1, label1), (image2, label2) in tqdm(itertools.product(images, images), total=(n_people*n_images_per_person)**2):
    batch_images1 = load_image_batch([image1])
    batch_images2 = load_image_batch([image2])
    
    with torch.no_grad():
        batch_features1 = model(batch_images1)
        batch_features2 = model(batch_images2)

    batch_features1 = Variable(batch_features1.squeeze())
    batch_features2 = Variable(batch_features2.squeeze())
    
    batch_code1 = cluster_model(batch_features1).cpu().data.numpy()
    batch_code2 = cluster_model(batch_features2).cpu().data.numpy()
    
    if label1==label2:
        if image1 != image2:
            l1 += np.linalg.norm(batch_code1 - batch_code2)
            c1 += 1
            l1p += np.linalg.norm(batch_features1.cpu().data.numpy() - batch_features2.cpu().data.numpy())
            c1p += 1
    else:
        l2 += np.linalg.norm(batch_code1 - batch_code2)
        c2 += 1
        l2p += np.linalg.norm(batch_features1.cpu().data.numpy() - batch_features2.cpu().data.numpy())
        c2p += 1
        
print(l1/c1)
print(l2/c2)
print(l1/c1 / (l2/c2))

print(l1p/c1p)
print(l2p/c2p)
print((l1p/c1p)/(l2p/c2p))

In [ ]:
print(19/25.

In [ ]:
200/287.

In [ ]:
imgs = []
n=4
s=400
for i in range(s, s+n):
    filename = images[i][0]

    img = PIL.Image.open(filename)
    img = torchvision.transforms.Resize(256)(img)
    img = torchvision.transforms.RandomCrop(224)(img)
    img = torchvision.transforms.RandomGrayscale(p=0.2)(img)
    imgs += [img]
    
new_im = PIL.Image.new('RGB', (224*n, 224))

x_offset = 0
for im in imgs:
    new_im.paste(im, (x_offset,0))
    x_offset += im.size[0]

new_im